# Population to netCDF

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import rasterio

from config import DATA_SRC, POP_DATA_SRC

In [2]:
population_grid_2015_file = (POP_DATA_SRC / 'nasa_grid' /'population_2015_halfres.tif')

In [3]:
with rasterio.open(str(population_grid_2015_file)) as pop:
    print(pop.meta)
    pop_meta = pop.meta
    trns = pop.transform
    population = pop.read(1)
    
population = population.T

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 21600, 'height': 8700, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': Affine(0.01666666666666678, 0.0, -180.0,
       0.0, -0.01666666666666678, 85.0000000000092)}


In [10]:
pop_x = np.arange(0, population.shape[0])
pop_y = np.arange(0, population.shape[1])

# Not totally conviced this is right, might prefer to re implement...
pop_lon, pop_lat = trns * np.meshgrid(pop_y, pop_x)

In [12]:
population.shape = (*population.shape, 1)

In [14]:
ds = xr.Dataset({'population': (['x', 'y', 'time'],  population)},
                 coords={'lon': (['x', 'y'], pop_lon),
                       'lat': (['x', 'y'], pop_lat),
                       'time': pd.date_range('2015', periods=1, freq='A-DEC')
                        })

In [15]:
ds.to_netcdf(str(POP_DATA_SRC / 'nasa_grid' / 'population2.nc'),format='NETCDF4', 
             encoding={'population': {
                                 'zlib': True,
                                 '_FillValue': -9999
                             },
                       'lon': {
                                 'zlib': True,
                                 '_FillValue': -9999
                             },
                       'lat': {
                                 'zlib': True,
                                 '_FillValue': -9999
                             }
                      }
            )

In [25]:
ds.population[0,0]

<xarray.DataArray 'population' (time: 1)>
array([ 0.], dtype=float32)
Coordinates:
    lon      float64 -180.0
    lat      float64 85.0
  * time     (time) datetime64[ns] 2015-12-31